## Альтернативный подход к задачам мультиклассовой классификации:

Вместо того, чтобы предсказывать таргет напрямую, мы можем закодировать таргет в двоичном коде и предсказывать каждый бит по отдельности.

Тогда нам понадобится всего $~ \text{log}_2(L + 1)$ классификаторов на L + 1 класс.

Попробуем обучить набор таких логрегов и сравнить качество полученного классификатора с мультиномиальной и OvR регрессиями.

In [1]:
# импорт базовых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set();
import scipy.stats as sps
from sklearn.datasets import fetch_openml

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#### Шаг 0: импорт и препроцессинг данных

In [3]:
%%time
mnist = fetch_openml(data_id=554) # https://www.openml.org/d/554
# генерируем сегментирующую случайную переменую
rn = pd.Series(sps.randint.rvs(1, 101, size = len(mnist.data), random_state = 42))
# разбиваем на трейн/валидацию/тест
X = mnist.data
y = mnist.target
train_mask, val_mask, test_mask = (rn <= 60), ((rn > 60) & (rn <= 70)), (rn > 70)
X_train, y_train, X_test, y_test, X_val, y_val = X[train_mask], y[train_mask], X[test_mask], y[test_mask], X[val_mask], y[val_mask]

CPU times: user 55.4 s, sys: 1min 22s, total: 2min 17s
Wall time: 2min 50s


In [4]:
# нормируем данные
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

#### Шаг 1: учим классификаторы с семинара

Вообще говоря, мы можем научить классификаторы с базовыми гиперпараметрами -- на семинаре мы видели, что они показывают на нашей задаче неплохое качество, но ведь нет предела совершенству -- давайте подберём какие-нибудь гиперпараметры для логрега (список можно посмотреть тут: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

#### Автоматизировать подбор гиперпараметров , как правило, удобнее -- ручной подбор предпочтителен для понимания, что и как влияет на качество моделей, но обычно занимает слишком много времени и сил без каких-либо преимуществ над автоматическим отбором

Реализуем функцию grid_search, которая будет делать следующее: принимая обучающую и валидационную выборки, функция обучает набор из классификаторов со всевозможными комбинациями гиперпараметров (предварительно указанных нами для перебора в словаре) и выбирает из них тот, чьё качество по целевой метрике на валидационной выборке оказывается наилучшим.

$\textbf{Вопрос для размышления:}$ можно ли тут попробовать "встроить" защиту от переобучения?

In [5]:
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

#реализуем функцию для подбора гиперпараметров модели
#для перехода от словаря параметров к списку комбинаций может быть полезен sklearn.model_selection.ParameterGrid
def grid_search(X_train, X_val, y_train, y_val, params_dict):
    '''
    Функция подбирает гиперпараметры мультиномиальной логитиеской регрессии для получения максимального значения accuracy
    на валидационной выборке, принимает:
    X_train -- DataFrame независимых переменных на обучающей выборке
    X_val -- DataFrame независимых переменных на валидационной выборке
    y_train -- Series таргета на обучающей выборке
    y_val -- Series таргета на валидационной выборке
    params_dict -- словарь гиперпараметров в формате {'paramater_nm':[value_1, value_2, ...]}
    '''
    best_score = 0
    i = 0
    for collection in ParameterGrid(params_dict):
        clf = LogisticRegression(**collection,
                                 multi_class='multinomial',
                                 max_iter=1000,
                                 fit_intercept=True)

        clf.fit(X_train, y_train)
        
        score = roc_auc_score(y_val, clf.predict_proba(X_val), average='macro', multi_class = 'ovr')
        score1 = accuracy_score(y_val, clf.predict(X_val))
        if best_score < score:
            best_score = score
            best_parametrs = collection

        print(i,collection,'Score:', score,'Best_score:',best_score,'Accuracy:',score1)
        i += 1
    return (best_parametrs)  #ну или best_parameters, если вдруг так интереснее

Сначала я подбирал параметры с большим размахом, а потом выделил интервал [0.005:0.05], на котором результат был лучше всего и рассмотрел его более подробно

In [210]:
# тут обучаем свой классификатор -- можно просто .fit() без подбора параметров, можно -- с подбором
grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100],
    
}
grid_search(
    X_train,
    X_val,
    y_train,
    y_val,
    grid
)

In [216]:
grid = {
    'C': np.linspace(0.001,0.05,10),
    
}
grid_search(
    X_train,
    X_val,
    y_train,
    y_val,
    grid
)

Одним из лучших по roc и accuracy оказался параметр C = 0.02 и roc = 0.993 , acc = 0.92

#### Шаг 2: бинаризуем таргет и учим классификаторы

In [75]:
def num_to_bin(x):
    x = int(x)
    b = []
    while x > 0:
        b = [x % 2] + b
        x = x // 2
    for i in range(4 - len(b)):
        b.insert(0, 0)
    return np.array(b)


def make_binary_predictors(y):
    '''
    Функция принимает Series y c категориальной переменной и делает DataFrame с [log_2(L+1)] столбцами из 0 и 1
    
    подсказка: в нашем конкретном случае можно переводить десятичное число в двоичное 
    '''

    targets = pd.DataFrame(np.concatenate(y.astype('object').apply(num_to_bin).values).reshape(-1,4))
    targets.columns = ['tar0', 'tar1', 'tar2', 'tar3']
    return (targets)

In [219]:
def f_norm(x):
    x = list(x)
    s = 0
    for i in range(len(x)):
        s += x[i]
    for i in range(len(x)):
        x[i] /= s
    return x
def softmax(x):
    
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

class BinarisedTargetClassifier():
    '''
    класс BinarisedTargetClassifier -- мультиклассовый классификатор на основании нескольких бинарных логистических регрессий
    '''

    def __init__(self):
        pass

    def fit(self,X_train, y_train):

        self.clf0 = LogisticRegression(fit_intercept=True, max_iter=1000)
        self.clf1 = LogisticRegression(fit_intercept=True, max_iter=1000)
        self.clf2 = LogisticRegression(fit_intercept=True, max_iter=1000)
        self.clf3 = LogisticRegression(fit_intercept=True, max_iter=1000)

        self.clf0.fit(X_train, y_train['tar0'])
        self.clf1.fit(X_train, y_train['tar1'])
        self.clf2.fit(X_train, y_train['tar2'])
        self.clf3.fit(X_train, y_train['tar3'])

    def predict(self,X):
        predictions = (lambda x: 8*x[0] + 4*x[1] + 2*x[2] + x[3])(np.concatenate([
            self.clf0.predict(X),
            self.clf1.predict(X),
            self.clf2.predict(X),
            self.clf3.predict(X)
        ]).transpose().reshape(4,-1))
        
        return list(map(str,list(predictions)))

    def predict_score(self,X):  #(без него не построить AUC, но в целом обойтись можно)
        end_preds = np.array([])
        
        preds = [
        self.clf0.predict_proba(X),
        self.clf1.predict_proba(X),
        self.clf2.predict_proba(X),
        self.clf3.predict_proba(X)]
        
        for x in ['0000','0001','0010','0011','0100','0101','0110','0111','1000','1001']:
            ptr = np.ones(X.shape[0])
            t = 0
            for i in x:
                ptr = ptr * preds[t][:,int(i)]
                t+=1
            end_preds = np.concatenate([end_preds,ptr])
            
        end_preds = end_preds.reshape(-1,10)
        end_preds = np.apply_along_axis(softmax , 1, end_preds)
        return end_preds

In [221]:
#учим созданный классификатор
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

clf_bin = BinarisedTargetClassifier()
#обучаем
clf_bin.fit(X_train, make_binary_predictors(y_train))
#предсказываем класс, считаем accuracy
y_pred = clf_bin.predict(X_test)
accuracy_bin = accuracy_score(y_test, y_pred)
#предсказываем вероятность, считаем AUC
score_bin = clf_bin.predict_score(X_test)
auc_bin = roc_auc_score(y_test, score_bin, average='macro', multi_class = 'ovr')

#### Шаг 3: сравнение качества полученных классификаторов

In [227]:
clf = LogisticRegression(multi_class='multinomial',
                         max_iter=1000,
                         fit_intercept=True,
                         C=0.02)

In [228]:
clf.fit(X_train,y_train)

LogisticRegression(C=0.02, max_iter=1000, multi_class='multinomial')

In [229]:
accuracy_test = clf.score(X_test, y_test)
y_score = clf.predict_proba(X_test)
auc_test = roc_auc_score(y_test, y_score, average='macro', multi_class = 'ovr')

In [232]:
res = pd.DataFrame(
{'regression_type' : ['sample','bin'],
'accuracy' : [accuracy_test,accuracy_bin],
'macro AUC' : [auc_test,auc_bin]
}
)
res

,regression_type,accuracy,macro AUC
0,sample,0.921090,0.992705
1,bin,0.711953,0.502715


$\textbf{Вывод}:$

Мультиномиальная регргрессия показала значительно лучший результат чем побитовая, accuracy = 0.921  и auc = 0.992 против accuracy = 0.71 и auc = 0.5. Как мне кажется, такой исход был очевиден, потому ,что когда мы бинаризуем target, мы устанавливаем некотурую взаимосвязь между категориями, хотя очевидно, что зависимости значения числа от его написания нет, тем самым мы усложняем задачу для нашей модели из-за того, что накладываем на неё неправильные условия. 